<a href="https://colab.research.google.com/github/keshavkmr48/Kaggle/blob/master/NLP_DISASTER_TWEET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setting up kaggle API to download competition dataset**

In [1]:
# !pip install kaggle --upgrade
# ! mkdir .kaggle
# import json
# token = {"username":"keshavkmr48","key":"014d22aa9430a95daaf028ec3e3463a2"}
# with open('/content/.kaggle/kaggle.json','w') as file:
#   json.dump(token,file)
#  !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
#  !kaggle config set -n path -v{/content}
#  !chmod 600 /root/.kaggle/kaggle.json
# download competition data
!kaggle competitions download -c nlp-getting-started

train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from xgboost import plot_importance
from sklearn import ensemble
# from catboost import CatBoostRegressor
from matplotlib import pyplot
# import shap
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
import gc, os
import json
import string
from wordcloud import STOPWORDS
from sklearn import preprocessing
from sklearn import metrics
import joblib
pd.set_option('display.max_columns', 1000)


In [3]:
# list data in the downloaded folder
#os.listdir('{/content}/competitions/data-science-bowl-2019')
os.chdir('{/content}/competitions/nlp-getting-started')
os.listdir()

['train_folds.csv', 'train.csv', 'test.csv', 'sample_submission.csv']

In [0]:
def read_data():
    print('Reading train.csv file....')
    train = pd.read_csv('train.csv')
    print('Training.csv file have {} rows and {} columns'.format(train.shape[0], train.shape[1]))

    print('Reading test.csv file....')
    test = pd.read_csv('test.csv')
    print('Test.csv file have {} rows and {} columns'.format(test.shape[0], test.shape[1]))


    print('Reading sample_submission.csv file....')
    sample_submission = pd.read_csv('sample_submission.csv')
    print('Sample_submission.csv file have {} rows and {} columns'.format(sample_submission.shape[0], sample_submission.shape[1]))
    return train, test, sample_submission

In [5]:
train, test, sample_submission = read_data()

Reading train.csv file....
Training.csv file have 7613 rows and 5 columns
Reading test.csv file....
Test.csv file have 3263 rows and 4 columns
Reading sample_submission.csv file....
Sample_submission.csv file have 3263 rows and 2 columns


In [0]:
# from multiprocessing import Pool

# def json_loader(x):
#   return json.loads(x)
# pool = Pool(4)
# result = pool.map_async(json_loader,train.event_data)
# df = pd.DataFrame(list(result.get()))

In [6]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
if __name__ == "__main__":
    train["kfold"] = -1

    train = train.sample(frac=1).reset_index(drop=True)

    kf = StratifiedKFold(n_splits=5, shuffle=False, random_state=42)


    for fold, (train_idx, val_idx) in enumerate(kf.split(X=train, y=train.target.values)):
        print(len(train_idx), len(val_idx))
        train.loc[val_idx, 'kfold'] = fold
    

    train.to_csv("train_folds.csv", index=False)

6089 1524
6090 1523
6091 1522
6091 1522
6091 1522


In [0]:
MODELS = {
    "randomforest": ensemble.RandomForestClassifier(n_estimators=200, n_jobs=-1, verbose=2),
    "extratrees": ensemble.ExtraTreesClassifier(n_estimators=200, n_jobs=-1, verbose=2),
}
del train
TRAINING_DATA = pd.read_csv('train_folds.csv')

In [0]:
FOLD_MAPPPING = {
    0: [1, 2, 3, 4],
    1: [0, 2, 3, 4],
    2: [0, 1, 3, 4],
    3: [0, 1, 2, 4],
    4: [0, 1, 2, 3]
}

In [0]:
# word2vec embedding extraction
def text_processor (text_value):
  text = 'ram'
  return text

In [0]:
# feature_engineering on text column

# char_count
TRAINING_DATA['char_count'] = TRAINING_DATA['text'].apply(lambda x: len(str(x)))
test['char_count'] = test['text'].apply(lambda x: len(str(x)))

# punctuation_count
TRAINING_DATA['punctuation_count'] = TRAINING_DATA['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test['punctuation_count'] = test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# hashtag_count
TRAINING_DATA['hashtag_count'] = TRAINING_DATA['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
test['hashtag_count'] = test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# mention_count
TRAINING_DATA['mention_count'] = TRAINING_DATA['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
test['mention_count'] = test['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

# unique_word_count
TRAINING_DATA['unique_word_count'] = TRAINING_DATA['text'].apply(lambda x: len(set(str(x).split())))
test['unique_word_count'] = test['text'].apply(lambda x: len(set(str(x).split())))

# stop_word_count
TRAINING_DATA['stop_word_count'] = TRAINING_DATA['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))
test['stop_word_count'] = test['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))

# url_count
TRAINING_DATA['url_count'] = TRAINING_DATA['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
test['url_count'] = test['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# mean_word_length
TRAINING_DATA['mean_word_length'] = TRAINING_DATA['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test['mean_word_length'] = test['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
# word_count
TRAINING_DATA['word_count'] = TRAINING_DATA['text'].apply(lambda x: len(str(x).split()))
test['word_count'] = test['text'].apply(lambda x: len(str(x).split()))

In [0]:
# Missing value filler 
TRAINING_DATA.keyword = TRAINING_DATA.keyword.fillna('No_Location')
TRAINING_DATA.location = TRAINING_DATA.location.fillna('No_Location')
test.keyword = test.keyword.fillna('No_Key')
test.location = test.location.fillna('No_Location')

In [16]:
TRAINING_DATA.head()

,id,keyword,location,text,target,kfold,char_count,punctuation_count,hashtag_count,mention_count,unique_word_count,stop_word_count,url_count,mean_word_length,word_count
0,9084,structural%20failure,"VÌ_sterÌ´s, Sweden",@whvholst @leashless And this is a structural ...,0,0,133,3,0,2,19,9,0,5.700000,20
1,10052,twister,Long Island,@mrsbinker @EmilioRivera48 @davidlabrava Mine ...,0,0,137,4,0,3,22,11,0,5.272727,22
2,8232,riot,No_Location,To All The Meat-Loving Feminists Of The World ...,0,0,105,6,0,0,13,6,1,6.571429,14
3,5642,flooding,?,@themaine is it too soon to start flooding you...,0,0,105,4,0,1,18,8,0,4.578947,19
4,6619,inundated,"Toronto, ON, Canada",inundated with Westeros Between Storm of Sword...,0,0,144,4,0,0,24,13,0,4.370370,27


In [0]:
if __name__ == "__main__":

  for MODEL in MODELS.keys():
    for FOLD in range(5):
      train_df = TRAINING_DATA[TRAINING_DATA.kfold.isin(FOLD_MAPPPING.get(FOLD))].reset_index(drop=True)
      valid_df = TRAINING_DATA[TRAINING_DATA.kfold==FOLD].reset_index(drop=True)

      ytrain = train_df.target.values
      yvalid = valid_df.target.values

      train_df = train_df.drop(["id", "target", "kfold"], axis=1)
      valid_df = valid_df.drop(["id", "target", "kfold"], axis=1)

      train_df.text = train_df.text.apply(lambda x : text_processor(x))
      valid_df.text = valid_df.text.apply(lambda x : text_processor(x))

      valid_df = valid_df[train_df.columns]

      # label_encoders = {}
      # for c in train_df.columns:
      #     lbl = preprocessing.LabelEncoder()
      #     lbl.fit(train_df[c].values.tolist() + valid_df[c].values.tolist() + df_test[c].values.tolist())
      #     train_df.loc[:, c] = lbl.transform(train_df[c].values.tolist())
      #     valid_df.loc[:, c] = lbl.transform(valid_df[c].values.tolist())
      #     label_encoders[c] = lbl
      
      # data is ready to train
      clf = MODELS[MODEL]
      clf.fit(train_df, ytrain)
      preds = clf.predict_proba(valid_df)[:, 1]
      print(metrics.roc_auc_score(yvalid, preds))

      # joblib.dump(label_encoders, f"models/{MODEL}_{FOLD}_label_encoder.pkl")
      joblib.dump(clf, f"models/{MODEL}_{FOLD}.pkl")
      joblib.dump(train_df.columns, f"models/{MODEL}_{FOLD}_columns.pkl")
